Data design:
This code is for basic data design and cleaning. The objective is to generate three tables from the raw full dataset-JSON data. 

We plan to generate three tables from the raw JSON data:

1. Researcher table: researcher_df 
This table contains one row per researcher.
Required fields include: 
"researcher_id": a unique identifier for each researcher; 
"first_name"; 
"last_name"; 
"email"; 
"title"; 
"overview":
"department_raw" (parsed from the title directly before cleaning); 
"role": the role of this research in Duke University(for example, professor, assistant professor, graduate student, etc.)The role field for each researcher is determined through a two-stage extraction pipeline. The general principle is: use the primaryAppointment.title first; if that fails, fall back to parsing the overview text. All matching is case-insensitive unless otherwise specified.
pub_count: the overall amounts of publication of this researcher;
latest_pub_date: the newest publication date.


2. Publication table: publication_clean_df 
****** There are a total of 481,572 author–publication rows, but the number of orginal pub_id(pulled out directly from duke database)'s  values is also 481,572. This means that each pub_id appears only once, and there is no case where two authors share the same pub_id. Therefore, pub_id cannot be used as an indicator of co-authorship, because it does not represent shared publications among multiple researchers. 

****** In this case, use "title to create unique_ids and coauthor pairs. (After testing, there is no missing in titles)

this table contains one row per publication.
Required fields include:

"pub_unique_id": the unique_id we created by titles. 
"researcher_id": researcher_id,
"researcher_full_name": first_name,
"pub_title": pub_title,
"pub_abstract": pub_abstract,
"pub_doi": pub_doi,
"duke_authors": All Duke-affiliated authors matched from the Duke researcher database.
"raw_pub_ids": All original pub_id values associated with this publication in the Duke source data.
"all_authors": all_authors, including people who are not matched via Duke database,
"pub_date": the date this research was published. 


3. Coauthor relationship: coauthor_pairs_final  
This table contains one row per co-author pair. 
Based on the unique_ids we used "title" to create, we created the deduplicated author.
Required fields include:
Researcher_A_id,
Researcher_B_id,
Researcher_A_name,
Researcher_B_name,
joint_pub_numbers: how many publications they collaborated. 



In [ ]:
# packages
import pandas as pd
import numpy as np
import json
import requests
import os
!pip install nanoid
from nanoid import generate

In [ ]:
# import data
import json

with open("people_2025-11-25_13-57-52.json", "r") as f:
    data_dict = json.load(f)
data = data_dict["results"]

In [ ]:
# data strcuture
def print_json_structure(d, indent=0):
    prefix = "  " * indent
    if isinstance(d, dict):
        for k, v in d.items():
            print(f"{prefix}{k}: ({type(v).__name__})")
            print_json_structure(v, indent + 1)
    elif isinstance(d, list):
        print(f"{prefix}[list of length {len(d)}]")
        if len(d) > 0:
            print_json_structure(d[0], indent + 1)
    else:
        # primitive value -> 不展开
        print(f"{prefix}{d} ({type(d).__name__})")

print_json_structure(data)


In [ ]:
# need to figure out what the 'currentResearch' refers to.
for i in range(10):
    print(data[i]['currentResearch'])

In [ ]:
data[2]

In [ ]:
import random

random.seed(9999)

def stable_id():
    return hex(random.getrandbits(64))[2:]

# data is already a list of researchers
print(type(data), len(data))  # just to confirm

for person in data:
    person["unique_id"] = stable_id()


In [ ]:
# role extraction from "PrimaryAppionment[title]

import re

def get_role_phrase(raw_title: str):
    """Extract the role phrase before 'in/of/at' or comma."""
    if not raw_title:
        return None
    t = raw_title.strip()
    m = re.match(r'^(.*?)(?:\s+(?:in|of|at)\b|,)', t, flags=re.I)
    if m:
        return m.group(1).strip()
    return t


def normalize_role(raw_title: str):
    """Normalize academic role titles into a standard label."""
    if not raw_title:
        return None
    
    # Step 1: extract core role phrase
    t = get_role_phrase(raw_title)
    if not t:
        return None
    s = t.strip()

    # --- Matching rules in priority order ---

    # Emeritus/Emerita
    if re.search(r'\bProfessor\b.*\bEmerit', s, flags=re.I):
        return 'Professor Emeritus'

    # Professor of the Practice
    if re.search(r'\bProfessor\s+of\s+the\s+Practice\b', s, flags=re.I):
        return 'Professor of the Practice'
    if re.search(r'\bAssociate\s+Professor\s+of\s+the\s+Practice\b', s, flags=re.I):
        return 'Associate Professor of the Practice'
    if re.search(r'\bAssistant\s+Professor\s+of\s+the\s+Practice\b', s, flags=re.I):
        return 'Assistant Professor of the Practice'

    # Special prefixes
    if re.search(r'\bAdjunct\s+Professor\b', s, flags=re.I):
        return 'Adjunct Professor'
    if re.search(r'\bClinical\s+Professor\b', s, flags=re.I):
        return 'Clinical Professor'
    if re.search(r'\bResearch\s+Professor\b', s, flags=re.I):
        return 'Research Professor'
    if re.search(r'\bVisiting\s+Professor\b', s, flags=re.I):
        return 'Visiting Professor'
    
    # Medical titles
    if re.search(r'\bMedical\s+Assistant\s+Professor\b', s, flags=re.I):
        return 'Medical Assistant Professor'
    if re.search(r'\bMedical\s+Associate\s+Professor\b', s, flags=re.I):
        return 'Medical Associate Professor'
    if re.search(r'\bMedical\s+Professor\b', s, flags=re.I):
        return 'Medical Professor'

    # Standard professor ranks
    if re.search(r'\bAssociate\s+Professor\b', s, flags=re.I):
        return 'Associate Professor'
    if re.search(r'\bAssistant\s+Professor\b', s, flags=re.I):
        return 'Assistant Professor'
    if re.search(r'\bProfessor\b', s, flags=re.I):
        return 'Professor'

    # Clinical roles
    if re.search(r'\bClinical\s+Faculty\b', s, flags=re.I):
        return 'Clinical Faculty'
    if re.search(r'\bAssociate\s+Clinical\s+Professor\b', s, flags=re.I):
        return 'Associate Clinical Professor'
    if re.search(r'\bAssistant\s+Clinical\s+Professor\b', s, flags=re.I):
        return 'Assistant Clinical Professor'
    if re.search(r'\bClinical\s+Associate\b', s, flags=re.I):
        return 'Clinical Associate'
    if re.search(r'\bClinical\s+Affiliate\b', s, flags=re.I):
        return 'Clinical Affiliate'

    # Lecturer / Fellow
    if re.search(r'\bSenior\s+Lecturing\s+Fellow\b', s, flags=re.I):
        return 'Senior Lecturing Fellow'
    if re.search(r'\bLecturing\s+Fellow\b', s, flags=re.I):
        return 'Lecturing Fellow'
    if re.search(r'\bSenior\s+Lecturer\b', s, flags=re.I):
        return 'Senior Lecturer'
    if re.search(r'\bLecturer\b', s, flags=re.I):
        return 'Lecturer'

    # Instructor family
    if re.search(r'\bMedical\s+Instructor\b', s, flags=re.I):
        return 'Medical Instructor'
    if re.search(r'\bInstructor\b', s, flags=re.I):
        return 'Instructor'

    # In Residence / Artist in Residence / Scholar in Residence
    if re.search(r'\bExecutive\s+In\s+Residence\b', s, flags=re.I):
        return 'Executive in Residence'
    if re.search(r'\bArtist\s+In\s+Residence\b', s, flags=re.I):
        return 'Artist in Residence'
    if re.search(r'\bScholar\s+In\s+Residence\b', s, flags=re.I):
        return 'Scholar in Residence'

    # Fallback: return original phrase
    return s


In [ ]:
# Step 1: extract title from primaryAppointment
def extract_pa_title(pa):
    if isinstance(pa, dict):
        return pa.get("title")
    return None

# Step 2: add the new field to each researcher dict
for person in data:   # data is your full JSON list
    pa_title = extract_pa_title(person.get("primaryAppointment"))
    role = normalize_role(pa_title)  # use the function we defined earlier
    person["role"] = role


In [ ]:
# if the role is null, continue to fill the role with "overview"

import re

# 1. Regex pattern for roles in overview (case-insensitive)
role_pattern_overview = re.compile(
    r"("
    r"ph\.?\s*d\.?\s*candidate|"        # PhD Candidate / Ph.D. Candidate
    r"ph\.?\s*d\.?\s*student|"          # PhD Student / Ph.D. Student
    r"doctoral\s+student|"                # Doctoral Student
    r"master'?s?\s+student|"              # master's student / masters student
    r"m\.?\s*s\.?\s*student|"           # M.S. student / MS student
    r"master\s+student|"                  # Master student
    r"graduate\s+student|"                # Graduate student
    r"\bdirector\b"                      # Director
    r")",
    re.IGNORECASE
)

# 2. Fill role from overview ONLY when existing role is None
filled_from_overview = 0

for person in data:
    # Only try overview if role is currently None
    if person.get("role") is None:
        overview = person.get("overview") or ""
        
        # Search in overview text
        match = role_pattern_overview.search(overview)
        if match:
            # Use the matched string as role (keep original casing from overview)
            person["role"] = match.group(0).strip()
            filled_from_overview += 1

print("Number of roles filled from overview:", filled_from_overview)


In [ ]:
# department/major 
import re

def extract_department_from_title(primary_appointment):
    """
    primary_appointment is either:
      - None
      - dict like {"title": "Assistant Professor in ..."}
    """
    # 1. If no primaryAppointment
    if primary_appointment is None:
        return None
    
    # 2. Extract string from {"title": "..."} 
    if isinstance(primary_appointment, dict):
        title = primary_appointment.get("title", None)
    else:
        # fallback: if it's already a string
        title = primary_appointment
    
    if not isinstance(title, str):
        return None
    
    t = title.strip()
    if not t:
        return None

    lower = t.lower()

    # --- extraction rules ---
    idx_in = lower.rfind(" in ")
    if idx_in != -1:
        dept = t[idx_in + len(" in "):].strip()
    else:
        idx_of = lower.rfind(" of ")
        if idx_of != -1:
            dept = t[idx_of + len(" of "):].strip()
        else:
            return None

    # remove "the "
    dept = re.sub(r"^\s*the\s+", "", dept, flags=re.IGNORECASE)

    # remove prefixes
    dept = re.sub(r"^(Department|Division|School|Program|Centre|Center)\s+of\s+", "", dept, flags=re.IGNORECASE).strip()

    # Handle "Department of X" inside string
    m = re.search(r"Department\s+of\s+(.+)", dept, flags=re.IGNORECASE)
    if m:
        dept = m.group(1).strip()

    # Handle "School of Nursing" → "Nursing"
    m2 = re.search(r"School\s+of\s+(.+)", dept, flags=re.IGNORECASE)
    if m2:
        dept = m2.group(1).strip()

    return dept if dept else None


# --- Attach to your data ---
for person in data:
    person["department_raw"] = extract_department_from_title(person.get("primaryAppointment"))


In [ ]:

# generate table one: researcher table 
import pandas as pd

rows = []

for person in data:   # data is your list of 11547 researchers

    # 1. unique researcher id
    rid = person.get("unique_id")

    # 2. name fields
    first = person.get("firstName")
    last = person.get("lastName")
    full = f"{first} {last}".strip() if first or last else None

    # 3. email
    email = person.get("email")

    # 4. primary appointment title
    pa = person.get("primaryAppointment")
    if isinstance(pa, dict):
        title = pa.get("title")
    else:
        title = None

    # 5. department_raw from your earlier extraction
    dept = person.get("department_raw")

    # 6. role (you already extracted earlier)
    role = person.get("role")

    # 7. publication count
    pubs = person.get("publications", {})
    pub_count = pubs.get("count", 0) if isinstance(pubs, dict) else 0

     # 8. latest publication date (take max over all publicationDate.date)
    latest_pub = None
    if isinstance(pubs, dict):
        results = pubs.get("results", [])
        dates = []
        for p in results:
            pub_info = p.get("publication", {}) or {}
            pub_date_obj = pub_info.get("publicationDate") or {}
            date_str = pub_date_obj.get("date")   # e.g. "2025-02-01"
            if date_str:
                dates.append(date_str)
        if dates:
            latest_pub = max(dates)

    

    #9. overview
    overview = person.get("overview")

    # add row
    rows.append({
        "researcher_id": rid,
        "first_name": first,
        "last_name": last,
        "full_name": full,
        "email": email,
        "title": title,
        "department_raw": dept,
        "role": role,
        "overview": overview,
        "pub_count": pub_count,
        "latest_pub_date": latest_pub,
    
    })

# convert to DataFrame
researcher_df = pd.DataFrame(rows)

researcher_df.head()


In [ ]:
#generate table two: publication table 
# step one - original publication-author  （include duplicated author-pub relationships)
import pandas as pd
import unicodedata
import re

def clean_title(title):
    """
    Clean publication titles for consistency.
    - Lowercase
    - Strip whitespace
    - Normalize unicode
    - Remove repeated spaces
    - Remove trailing punctuation (.,;:)
    """

    if title is None:
        return None
    
    # Normalize unicode (fix accents, weird characters)
    title = unicodedata.normalize("NFKC", title)

    # Lowercase
    title = title.lower()

    # Remove extra spaces
    title = re.sub(r"\s+", " ", title).strip()

    # Remove trailing punctuation such as ".", ";", ":" 
    title = re.sub(r"[.;:]+$", "", title)

    return title


pub_rows = []

for person in data:  # data is your list of researchers

    # researcher basic info
    researcher_id = person.get("unique_id")
    first = person.get("firstName")
    last = person.get("lastName")
    full_name = f"{first} {last}".strip() if first or last else None
    
    # publications block
    pubs = person.get("publications", {})
    if isinstance(pubs, dict):
        results = pubs.get("results", [])
    else:
        results = []
    
    # loop over each publication for this researcher
    for p in results:
        raw_pub_id = p.get("id")
        pub_info = p.get("publication", {}) or {}
        
        pub_title_raw = pub_info.get("title")
        pub_title_clean = clean_title(pub_title_raw)

        pub_abstract = pub_info.get("abstract")
        pub_doi = pub_info.get("doi")
        
        all_authors_obj = pub_info.get("allAuthors", {}) or {}
        all_authors = all_authors_obj.get("fullList")

        # extract publication date
        date_obj = pub_info.get("publicationDate", {}) or {}
        pub_date = date_obj.get("date")

        # append row
        pub_rows.append({
            "raw_pub_id": raw_pub_id,
            "researcher_id": researcher_id,
            "full_name": full_name,
            "pub_title_raw": pub_title_raw,
            "pub_title_clean": pub_title_clean,
            "pub_abstract": pub_abstract,
            "pub_doi": pub_doi,
            "all_authors": all_authors,
            "pub_date": pub_date
        })

# build the publication DataFrame
raw_pub_df = pd.DataFrame(pub_rows)

### step 2 deduplicate author-pub rows 

import pandas as pd

# Make a copy so we don't modify raw_pub_df in place
dedup_pub_df = raw_pub_df.copy()

# Parse pub_date as datetime (invalid or missing dates become NaT)
dedup_pub_df["pub_date_parsed"] = pd.to_datetime(
    dedup_pub_df["pub_date"], errors="coerce"
)

# Helper flag: True if this row has a valid date, False otherwise
dedup_pub_df["has_date"] = dedup_pub_df["pub_date_parsed"].notna()

# Sort so that:
# - same (researcher_id, pub_title_clean) group
# - rows with a date come BEFORE rows without date
# - among rows with dates, the LATEST date comes first
dedup_pub_df = dedup_pub_df.sort_values(
    by=["researcher_id", "pub_title_clean", "has_date", "pub_date_parsed"],
    ascending=[True, True, False, False]  # keep best row at the top of each group
)

# Now drop duplicates: keep the first row in each (researcher_id, pub_title_clean) group
dedup_pub_df = dedup_pub_df.drop_duplicates(
    subset=["researcher_id", "pub_title_clean"],
    keep="first"
)

# Drop helper columns
dedup_pub_df = dedup_pub_df.drop(columns=["pub_date_parsed", "has_date"])

# Optional: check how many rows were removed
print("Original rows:", raw_pub_df.shape[0])
print("After de-duplicating by (researcher_id, pub_title_clean):", dedup_pub_df.shape[0])

raw_pub_df = dedup_pub_df



In [ ]:
#generate table two: publication table 
import pandas as pd
import random

# Use global random seed for reproducible IDs
random.seed(999999)

def stable_id():
    # Generate a reproducible pseudo-random hex string ID
    return hex(random.getrandbits(64))[2:]


clean_pub = raw_pub_df.dropna(subset=["pub_title_clean"]).copy()
clean_pub = clean_pub[clean_pub["pub_title_clean"].astype(str).str.strip() != ""]

grouped = clean_pub.groupby("pub_title_clean")

new_pub_rows = []

for title_clean, group in grouped:
    
    # Stable ID for this publication (one per cleaned title)
    pub_unique_id = stable_id()
    
    # Unique Duke authors in this group
    duke_authors = "; ".join(group["full_name"].dropna().unique())
    
    # All raw_pub_id values in this group
    raw_pub_ids = list(group["raw_pub_id"].dropna().unique())
    
    # Representative original (raw) title
    pub_title_raw = group["pub_title_raw"].iloc[0]
    
    # First non-null values for abstract, doi, date
    pub_abstract = group["pub_abstract"].iloc[0]
    pub_doi = group["pub_doi"].iloc[0]
    pub_date = group["pub_date"].iloc[0]
    
    new_pub_rows.append({
        "pub_unique_id": pub_unique_id,
        "pub_title_raw": pub_title_raw,
        "pub_title_clean": title_clean,
        "duke_authors": duke_authors,
        "raw_pub_ids": raw_pub_ids,
        "pub_abstract": pub_abstract,
        "pub_doi": pub_doi,
        "pub_date": pub_date
    })

publication_clean_df = pd.DataFrame(new_pub_rows)
publication_clean_df.head()



In [ ]:
## create coauthorship table 
# step one (create a table one row per pub_id + researcher pairs (not dedepulicated) 
from itertools import combinations
import pandas as pd

author_pub = raw_pub_df[[
    "pub_title_clean",
    "researcher_id",
    "full_name"
]].dropna(subset=["pub_title_clean", "researcher_id"])

author_pub = author_pub.merge(
    publication_clean_df[["pub_unique_id", "pub_title_clean"]],
    on="pub_title_clean",
    how="left"
)

print("Total author–pub rows:", len(author_pub))
print("Unique publication_unique_id:", author_pub["pub_unique_id"].nunique())

pairs = []

for uid, group in author_pub.groupby("pub_unique_id"):
    authors = group.to_dict("records")

    if len(authors) < 2:
        continue

    for a, b in combinations(authors, 2):

        if a["researcher_id"] < b["researcher_id"]:
            left, right = a, b
        else:
            left, right = b, a

        if left["researcher_id"] == right["researcher_id"]:
            continue

        pairs.append({
            "pub_unique_id": uid,
            "pub_title_cleaned":pub_title_clean,
            "researcher_A_id": left["researcher_id"],
            "researcher_B_id": right["researcher_id"],
            "researcher_A_name": left["full_name"],
            "researcher_B_name": right["full_name"]
        })

coauthor_pairs = pd.DataFrame(pairs)

print("Total raw coauthor pairs:", len(coauthor_pairs))
coauthor_pairs.head()


In [ ]:
coauthor_pairs.head()

In [ ]:
## create coauthorship table
# step two (created the deduplicated pairs with their joint pub counts)
coauthor_pairs_final = (
    coauthor_pairs
        .groupby(
            ["researcher_A_id", "researcher_B_id"],
            as_index=False
        )
        .agg({
            "pub_unique_id": "nunique",          
            "researcher_A_name": "first",        
            "researcher_B_name": "first"
        })
        .rename(columns={"pub_unique_id": "joint_publications"})
)



In [ ]:
coauthor_pairs_final.head(100)

In [ ]:
!pip install pyreadr
import pyreadr

pyreadr.write_rds("researcher_df.rds", researcher_df)
pyreadr.write_rds("publication_author_per_row", raw_pub_df)
pyreadr.write_rds("publication_clean_df.rds", publication_clean_df)
pyreadr.write_rds("coauthor_pairs_final.rds", coauthor_pairs_final)

